In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl
    import dolfinx
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        if arch != "complex":
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        else:
            !wget "https://fem-on-colab.github.io/releases/fenicsx-install-complex.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl
        import dolfinx

In [ ]:
try:
    import google.colab
except ImportError:
    pass
else:
    google.colab.output.enable_custom_widget_manager()
try:
    import pyvista
except ImportError:
    !pip3 install --upgrade itkwidgets pyvista
    import pyvista  # noqa: F401

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install --no-dependencies git+https://github.com/multiphenics/multiphenicsx.git
    import multiphenicsx  # noqa: F401

In [ ]:
# Download data files
!mkdir -p data
![ -f data/circle.h5 ] || wget https://github.com/multiphenics/multiphenicsx/raw/v0.2.0/tutorials/03_lagrange_multipliers/data/circle.h5 -O data/circle.h5
![ -f data/circle.xdmf ] || wget https://github.com/multiphenics/multiphenicsx/raw/v0.2.0/tutorials/03_lagrange_multipliers/data/circle.xdmf -O data/circle.xdmf

In [ ]:
import numpy as np
import mpi4py
import mpi4py.MPI
import petsc4py
import petsc4py.PETSc
import dolfinx.fem
import dolfinx.io
import dolfinx.mesh
import multiphenicsx.fem
import multiphenicsx.io
import ufl

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh(dolfinx.mesh.GhostMode.none)
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
cells_Omega1 = subdomains.indices[subdomains.values == 1]
cells_Omega2 = subdomains.indices[subdomains.values == 2]
facets_partial_Omega = boundaries.indices[boundaries.values == 1]
facets_Gamma = boundaries.indices[boundaries.values == 2]

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
multiphenicsx.io.plot_mesh_tags(subdomains)

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

In [ ]:
# Define associated measures
dx = ufl.Measure("dx")(subdomain_data=subdomains)
ds = ufl.Measure("ds")(subdomain_data=boundaries)
dS = ufl.Measure("dS")(subdomain_data=boundaries)
dS = dS(2)  # restrict to the interface, which has facet ID equal to 2

In [ ]:
# Define function spaces
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 2))
V1 = V.clone()
V2 = V.clone()
M = V.clone()

In [ ]:
# Define restrictions
dofs_V1_Omega1 = dolfinx.fem.locate_dofs_topological(V1, subdomains.dim, cells_Omega1)
dofs_V2_Omega2 = dolfinx.fem.locate_dofs_topological(V2, subdomains.dim, cells_Omega2)
dofs_M_Gamma = dolfinx.fem.locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V1_Omega1 = multiphenicsx.fem.DofMapRestriction(V1.dofmap, dofs_V1_Omega1)
restriction_V2_Omega2 = multiphenicsx.fem.DofMapRestriction(V2.dofmap, dofs_V2_Omega2)
restriction_M_Gamma = multiphenicsx.fem.DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V1_Omega1, restriction_V2_Omega2, restriction_M_Gamma]

In [ ]:
# Define trial and test functions
(u1, u2, l) = (ufl.TrialFunction(V1), ufl.TrialFunction(V2), ufl.TrialFunction(M))
(v1, v2, m) = (ufl.TestFunction(V1), ufl.TestFunction(V2), ufl.TestFunction(M))

In [ ]:
# Define problem block forms
zero = dolfinx.fem.Constant(mesh, petsc4py.PETSc.ScalarType(0))
a = [[ufl.inner(ufl.grad(u1), ufl.grad(v1)) * dx(1), None, ufl.inner(l("-"), v1("-")) * dS],
     [None, ufl.inner(ufl.grad(u2), ufl.grad(v2)) * dx(2), - ufl.inner(l("+"), v2("+")) * dS],
     [ufl.inner(u1("-"), m("-")) * dS, - ufl.inner(u2("+"), m("+")) * dS, None]]
f = [ufl.inner(1, v1) * dx(1), ufl.inner(1, v2) * dx(2), ufl.inner(zero, m("-")) * dS]
a_cpp = dolfinx.fem.form(a)
f_cpp = dolfinx.fem.form(f)

In [ ]:
# Define boundary conditions
dofs_V1_partial_Omega = dolfinx.fem.locate_dofs_topological(V1, boundaries.dim, facets_partial_Omega)
dofs_V2_partial_Omega = dolfinx.fem.locate_dofs_topological(V2, boundaries.dim, facets_partial_Omega)
bc1 = dolfinx.fem.dirichletbc(zero, dofs_V1_partial_Omega, V1)
bc2 = dolfinx.fem.dirichletbc(zero, dofs_V2_partial_Omega, V2)
bcs = [bc1, bc2]

In [ ]:
# Assemble the block linear system
A = multiphenicsx.fem.petsc.assemble_matrix_block(a_cpp, bcs=bcs, restriction=(restriction, restriction))
A.assemble()
F = multiphenicsx.fem.petsc.assemble_vector_block(f_cpp, a_cpp, bcs=bcs, restriction=restriction)

In [ ]:
# Solve
u1u2l = multiphenicsx.fem.petsc.create_vector_block(f_cpp, restriction=restriction)
ksp = petsc4py.PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, u1u2l)
u1u2l.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
(u1, u2, l) = (dolfinx.fem.Function(V1), dolfinx.fem.Function(V2), dolfinx.fem.Function(M))
with multiphenicsx.fem.petsc.BlockVecSubVectorWrapper(
        u1u2l, [V1.dofmap, V2.dofmap, M.dofmap], restriction) as u1u2l_wrapper:
    for u1u2l_wrapper_local, component in zip(u1u2l_wrapper, (u1, u2, l)):
        with component.vector.localForm() as component_local:
            component_local[:] = u1u2l_wrapper_local

In [ ]:
multiphenicsx.io.plot_scalar_field(u1, "u1")

In [ ]:
multiphenicsx.io.plot_scalar_field(u2, "u2")

In [ ]:
# Define trial and test functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [ ]:
# Define problem forms
a_ex = ufl.inner(ufl.grad(u), ufl.grad(v)) * dx
f_ex = ufl.inner(1, v) * dx

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_partial_Omega = dolfinx.fem.locate_dofs_topological(V, boundaries.dim, facets_partial_Omega)
bc_ex = dolfinx.fem.dirichletbc(zero, dofs_V_partial_Omega, V)

In [ ]:
# Solve
u_ex = dolfinx.fem.Function(V)
problem_ex = dolfinx.fem.petsc.LinearProblem(
    a_ex, f_ex, bcs=[bc_ex], u=u_ex,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
problem_ex.solve()
u_ex.vector.ghostUpdate(addv=petsc4py.PETSc.InsertMode.INSERT, mode=petsc4py.PETSc.ScatterMode.FORWARD)

In [ ]:
multiphenicsx.io.plot_scalar_field(u_ex, "u_ex")

In [ ]:
u_ex1_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(u_ex, u_ex) * dx(1))), op=mpi4py.MPI.SUM))
u_ex2_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(u_ex, u_ex) * dx(2))), op=mpi4py.MPI.SUM))
err1_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(u_ex - u1, u_ex - u1) * dx(1))), op=mpi4py.MPI.SUM))
err2_norm = np.sqrt(mesh.comm.allreduce(
    dolfinx.fem.assemble_scalar(dolfinx.fem.form(ufl.inner(u_ex - u2, u_ex - u2) * dx(2))), op=mpi4py.MPI.SUM))
print("Relative error on subdomain 1", err1_norm / u_ex1_norm)
print("Relative error on subdomain 2", err2_norm / u_ex2_norm)
assert np.isclose(err1_norm / u_ex1_norm, 0., atol=1.e-10)
assert np.isclose(err2_norm / u_ex2_norm, 0., atol=1.e-10)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/fenics -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi